In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [1]:
!pip install stable-baselines3[extra]

  Using cached gym-0.21.0-py3-none-any.whl
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2


In [2]:
!pip install pyglet==1.5.27

  Using cached pyglet-1.5.27-py3-none-any.whl (1.1 MB)
  Attempting uninstall: pyglet
    Found existing installation: pyglet 1.3.2
    Uninstalling pyglet-1.3.2:
      Successfully uninstalled pyglet-1.3.2


In [3]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [4]:
environment_name = "CartPole-v0"

In [5]:
env = gym.make(environment_name)

In [10]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:12.0
Episode:2 Score:20.0
Episode:3 Score:11.0
Episode:4 Score:45.0
Episode:5 Score:21.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [11]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [12]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 3.6562735e-01, -1.7966382e+38, -1.9491954e-01,  1.0251777e+38],
      dtype=float32)

# 3. Train an RL Model

In [13]:
log_path = os.path.join('Training', 'Logs')
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [21]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 3107 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1932         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0055764127 |
|    clip_fraction        | 0.0553       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.488       |
|    explained_variance   | 0.139        |
|    learning_rate        | 0.0003       |
|    loss                 | 0.00461      |
|    n_updates            | 210          |
|    policy_gradient_loss | -0.00539     |
|    value_loss           | 0.0845       |
----------------------------

# 4. Save and Reload Model

In [16]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [17]:
model.save(PPO_path)

In [18]:
del model

In [20]:
model = PPO.load(PPO_path, env=env)

# 5. Evaluation

In [22]:
from stable_baselines3.common.evaluation import evaluate_policy

In [23]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\Users\Andrew\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [24]:
env.close()

# 6. Test Model

In [25]:
obs = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([ 0.08271974,  0.03700212, -0.05591203, -0.06682089], dtype=float32)}]


In [26]:
env.close()

# 7. Viewing Logs in Tensorboard

In [27]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [29]:
!tensorboard --logdir={training_log_path}

^C


# 8. Adding a callback to the training Stage

In [30]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [32]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [31]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [33]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [34]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [35]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 3176 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1959         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0073216455 |
|    clip_fraction        | 0.0887       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.687       |
|    explained_variance   | -0.00473     |
|    learning_rate        | 0.0003       |
|    loss                 | 6.48         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0139      |
|    value_loss           | 60.9         |
----------------------------

In [39]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [40]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [41]:
env.close()

# 9. Changing Policies

In [42]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [43]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [44]:
model.learn(total_timesteps=20000, callback=eval_callback)

-----------------------------
| time/              |      |
|    fps             | 2645 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1444        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014456979 |
|    clip_fraction        | 0.188       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.682      |
|    explained_variance   | -0.000261   |
|    learning_rate        | 0.0003      |
|    loss                 | 3.57        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0232     |
|    value_loss           | 21.2        |
-----------------------------------------
----------------------------------

# 10. Using an Alternate Algorithm

In [45]:
from stable_baselines3 import DQN

In [46]:
model = DQN('MlpPolicy', env, verbose = 1)

Using cpu device


In [47]:
model.learn(total_timesteps=20000)

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.966    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 14394    |
|    time_elapsed     | 0        |
|    total_timesteps  | 72       |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.911    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 16996    |
|    time_elapsed     | 0        |
|    total_timesteps  | 187      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.867    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 16467    |
|    time_elapsed     | 0        |
|    total_timesteps  | 280      |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 112      |
|    fps              | 17104    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2771     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 116      |
|    fps              | 17059    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2866     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 120      |
|    fps              | 17097    |
|    time_elapsed     | 0        |
|    total_timesteps  | 2958     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 220      |
|    fps              | 17382    |
|    time_elapsed     | 0        |
|    total_timesteps  | 5180     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 224      |
|    fps              | 17377    |
|    time_elapsed     | 0        |
|    total_timesteps  | 5248     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 228      |
|    fps              | 17351    |
|    time_elapsed     | 0        |
|    total_timesteps  | 5327     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 328      |
|    fps              | 17563    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7447     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 332      |
|    fps              | 17598    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7532     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 336      |
|    fps              | 17601    |
|    time_elapsed     | 0        |
|    total_timesteps  | 7604     |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 436      |
|    fps              | 17721    |
|    time_elapsed     | 0        |
|    total_timesteps  | 9995     |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 440      |
|    fps              | 17700    |
|    time_elapsed     | 0        |
|    total_timesteps  | 10089    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 444      |
|    fps              | 17667    |
|    time_elapsed     | 0        |
|    total_timesteps  | 10141    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 544      |
|    fps              | 17705    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12305    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 548      |
|    fps              | 17718    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12403    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 552      |
|    fps              | 17713    |
|    time_elapsed     | 0        |
|    total_timesteps  | 12541    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 652      |
|    fps              | 17654    |
|    time_elapsed     | 0        |
|    total_timesteps  | 15130    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 17653    |
|    time_elapsed     | 0        |
|    total_timesteps  | 15200    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 660      |
|    fps              | 17656    |
|    time_elapsed     | 0        |
|    total_timesteps  | 15291    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 760      |
|    fps              | 17648    |
|    time_elapsed     | 0        |
|    total_timesteps  | 17560    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 764      |
|    fps              | 17646    |
|    time_elapsed     | 1        |
|    total_timesteps  | 17647    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 768      |
|    fps              | 17644    |
|    time_elapsed     | 1        |
|    total_timesteps  | 17697    |
----------------------------------
----------------------------------
| rollout/          

----------------------------------
| rollout/            |          |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 868      |
|    fps              | 17666    |
|    time_elapsed     | 1        |
|    total_timesteps  | 19999    |
----------------------------------
